In [3]:
from helpers import *
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
pd.set_option('display.float_format', '{:.2f}'.format)

In [4]:
s3=s3_connection()

connection successful


### La motivation

In [5]:
regional_stat=s3.get_tables_from_s3("clichere/diffusion/final_comm.parquet")
print(regional_stat[["pop2022", "revmoy2022", "psup2022", "pcadr2022"]].describe())

        pop2022  revmoy2022  psup2022  pcadr2022
count  36632.00    34720.00  35222.00   35065.00
mean    1829.92    17049.74      0.27       0.12
std     9954.65     5194.78      0.17       0.13
min        0.00     3350.48      0.00       0.00
25%      182.00    14011.97      0.16       0.00
50%      428.00    16321.86      0.26       0.08
75%     1116.00    19124.01      0.37       0.18
max   875791.00   126903.91      1.00       1.00


In [7]:
df=s3.get_tables_from_s3("aayrapetyan/diffusion/final_table.parquet")
print(df[["pop2022", "revmoy2022", "psup2022", "pcadr2022"]].describe())

        pop2022  revmoy2022  psup2022  pcadr2022
count   2808.00     2808.00   2808.00    2808.00
mean   18078.54    18221.52      0.29       0.14
std    50290.90     6608.60      0.13       0.11
min       55.00     5818.04      0.00       0.00
25%     1570.25    14922.08      0.20       0.07
50%     4067.50    16931.46      0.28       0.12
75%    12203.00    19639.30      0.37       0.19
max   875791.00   107572.81      1.00       0.82


On voit que la population moyenne diminue lorsque l'on sélectionne les communes ayant une gare. Cela correspond au fait que les petites communes ont moins de chance d'avoir une gare. On passe de 36000 communes à 2800 communes, ce qui donne tout de même beaucoup d'observations. On remarque que le revenu moyen d'une commune en France est plus élevé lorsque l'on sélectionne seulement les communes comprenant une gare. De même, les personnes sont en moyenne plus diplômée et appartiennent aux catégories socio-professionnelles plus élevées également. 

On veut montrer qu'il existe un lien entre le revenu, la catégorie socio-professionnelle et le fait de prendre un train en France. Pour cela, on effectue une régression linéaire avec le nombre total de voyageurs comme variable dépendante. Notre but est d'expliquer pourquoi est-ce que cette variable est différente en fonction des communes, en contrôlant par la population de la commune. Les premières visualisations et les premières données suggèrent qu'il peut y avoir un lien entre son revenu et le fait de prendre un train.

### Création d'une nouvelle variable

In [8]:
df['freq_par_hab']= df['total_voyageurs_2022']/df['pop2022']
print(df[["freq_par_hab"]].head())

   freq_par_hab
0         18.09
1          4.52
2         90.82
3          0.88
4          1.40


In [9]:
df

,total_voyageurs_2022,nom_gare,uic,codeinsee,lon_gare,lat_gare,codecommune,nomcommune,pop2022,revmoy2022,...,pagri2022,pindp2022,pcadr2022,pempl2022,pouvr2022,pchom2022,dist_closest_station_km,nb_stations_same_commune,regions,freq_par_hab
0,40825,Abbaretz,87481614,44001,-1.52,47.55,44001,ABBARETZ,2257.00,12567.25,...,0.06,0.02,0.05,0.31,0.42,0.04,9.75,1,Pays de la Loire,18.09
1,177092,Achères Grand Cormier,87386052,78551,2.09,48.96,78551,SAINT-GERMAIN-EN-LAYE,39172.00,41601.99,...,0.00,0.04,0.39,0.23,0.07,0.06,1.97,1,Île-de-France,4.52
2,80648,Achiet-le-Grand,87342048,62005,2.78,50.13,62005,ACHIET-LE-GRAND,888.00,15813.03,...,0.00,0.00,0.11,0.16,0.38,0.08,4.22,1,Hauts-de-France,90.82
3,32800,Agay,87757559,83118,6.86,43.43,83118,SAINT-RAPHAEL,37114.00,26637.97,...,0.00,0.13,0.13,0.35,0.13,0.10,1.80,6,Provence-Alpes-Côte d'Azur,0.88
4,11325,Aigues-Mortes,87775858,30003,4.19,43.57,30003,AIGUES-MORTES,8076.00,20798.90,...,0.01,0.14,0.08,0.39,0.17,0.11,5.56,1,Occitanie,1.40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2803,176599,Weyersheim,87213678,67529,7.80,48.72,67529,WEYERSHEIM,3365.00,21737.75,...,0.01,0.15,0.23,0.22,0.07,0.03,2.10,1,Grand Est,52.48
2804,30849,Willer-sur-Thur,87182584,68372,7.07,47.84,68372,WILLER-SUR-THUR,1735.00,18620.66,...,0.00,0.09,0.09,0.24,0.34,0.04,1.82,1,Grand Est,17.78
2805,37020,Wimille - Wimereux,87317123,62894,1.61,50.76,62894,WIMILLE,4062.00,19524.12,...,0.01,0.07,0.15,0.21,0.30,0.06,4.08,1,Hauts-de-France,9.11
2806,38618,Ygos-Saint-Saturnin,87671487,40333,-0.74,43.98,40333,YGOS-SAINT-SATURNIN,1490.00,15004.68,...,0.00,0.08,0.11,0.33,0.32,0.09,6.68,1,Nouvelle-Aquitaine,25.92


In [10]:
communes_plus_1_gare = df[df['nb_stations_same_commune'] > 1]
# Nombre total de communes avec plus d'une gare
nombre_communes = len(communes_plus_1_gare)

# Afficher les résultats
print(f"Nombre de communes avec plus d'une gare : {nombre_communes}")
print(communes_plus_1_gare)

Nombre de communes avec plus d'une gare : 522
      total_voyageurs_2022                      nom_gare       uic codeinsee  \
3                    32800                          Agay  87757559     83118   
14                  586872                       Andrésy  87381491     78015   
18                16033835                    Argenteuil  87381848     95018   
19                   16320                    Argentière  87746859     74056   
32                   53740                     Babinière  87590349     44035   
...                    ...                           ...       ...       ...   
2781                 97742           Villaines-sous-Bois  87272021     95056   
2789               3855727  Villeparisis - Mitry-le-Neuf  87271510     77294   
2792                 53041           Villiers Montbarbin  87116749     77142   
2797               3770683               Vitry-sur-Seine  87545293     94081   
2802                 69296                        Wavrin  87286419     596

### Premières régressions

In [13]:
df['pnodip2022']=1-df['pbac2022']
df[['pbac2022', 'psup2022', 'pagri2022', 'pempl2022', 'pcadr2022', 'pchom2022', 'pindp2022', 'pouvr2022', 'pnodip2022']]=df[['pbac2022', 'psup2022', 'pagri2022', 'pempl2022', 'pcadr2022', 'pchom2022', 'pindp2022', 'pouvr2022', 'pnodip2022']]*100

In [15]:

X = df[['revmoy2022', 'pop2022', 'pnodip2022', 'psup2022', 'pcadr2022', 'pouvr2022', 'pchom2022', 'pempl2022', 'dist_closest_station_km']]
Y = df['total_voyageurs_2022']  

X = sm.add_constant(X)

model = sm.OLS(Y, X).fit()

print(model.summary())

                             OLS Regression Results                             
Dep. Variable:     total_voyageurs_2022   R-squared:                       0.146
Model:                              OLS   Adj. R-squared:                  0.143
Method:                   Least Squares   F-statistic:                     53.19
Date:                  Tue, 17 Dec 2024   Prob (F-statistic):           1.16e-89
Time:                          08:59:19   Log-Likelihood:                -45803.
No. Observations:                  2808   AIC:                         9.163e+04
Df Residuals:                      2798   BIC:                         9.169e+04
Df Model:                             9                                         
Covariance Type:              nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
const 